# Essential Data Sanity Checks

**Purpose**: Verify data integrity before feature engineering. Only the 3 critical checks needed.

1. **Data Integrity**: No missing IDs, consistent row counts
2. **Temporal Alignment**: Transaction dates within label timeline  
3. **Label Quality**: Distress rate and signal strength


In [1]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data")  # change if needed

customers = pd.read_csv(DATA_DIR / "customers.csv", parse_dates=["signup_date"])
transactions = pd.read_csv(DATA_DIR / "transactions.csv", parse_dates=["date"])
outcomes = pd.read_csv(DATA_DIR / "outcomes.csv", parse_dates=["distress_start_date", "event_date"])
daily_labels = pd.read_csv(DATA_DIR / "daily_labels.csv", parse_dates=["date"])

print(f"Loaded: {len(customers):,} customers, {len(transactions):,} transactions")

# Ensure daily_labels covers the full transaction window
tx_min, tx_max = transactions["date"].min(), transactions["date"].max()
full_dates = pd.date_range(tx_min, tx_max, freq="D")

grid = (
    customers[["customer_id"]]
    .assign(key=1)
    .merge(pd.DataFrame({"date": full_dates, "key":1}), on="key")
    .drop(columns="key")
)

# Join outcomes and recompute labels (7 days before event = 1)
grid = grid.merge(outcomes[["customer_id","event_date"]], on="customer_id", how="left")

win = 7
grid["label"] = (
    (grid["event_date"].notna()) &
    (grid["date"] >= grid["event_date"] - pd.Timedelta(days=win)) &
    (grid["date"] <= grid["event_date"])
).astype(int)

daily_labels = grid[["customer_id","date","label"]]

print(f"Regenerated daily_labels: {len(daily_labels):,} rows covering {tx_min.date()} → {tx_max.date()}")


Loaded: 2,500 customers, 1,122,312 transactions
Regenerated daily_labels: 465,000 rows covering 2025-02-16 → 2025-08-20


## 1. Data Integrity Check


In [2]:
# Check for missing customer IDs
missing_ids = customers["customer_id"].isna().sum() + transactions["customer_id"].isna().sum()
dupe_customers = customers["customer_id"].duplicated().sum()

# Foreign-key integrity: every tx customer_id exists in customers
valid_fk = transactions["customer_id"].isin(customers["customer_id"]).all()

integrity_pass = (missing_ids == 0) and (dupe_customers == 0) and valid_fk

print("🔍 DATA INTEGRITY")
print(f"- Missing customer_id values: {missing_ids}")
print(f"- Duplicate customer_id rows in customers: {dupe_customers}")
print(f"- Transactions reference valid customers: {valid_fk}")
print(f"✅ Data Integrity: {'PASS' if integrity_pass else 'FAIL'}")


🔍 DATA INTEGRITY
- Missing customer_id values: 0
- Duplicate customer_id rows in customers: 0
- Transactions reference valid customers: True
✅ Data Integrity: PASS


## 2. Temporal Alignment Check


In [3]:
# Convert is already parsed; compute ranges
tx_min, tx_max = transactions["date"].min(), transactions["date"].max()
lbl_min, lbl_max = daily_labels["date"].min(), daily_labels["date"].max()

within_timeline = (tx_min >= lbl_min) and (tx_max <= lbl_max)

# Outcomes present should fall inside the label span
events = outcomes["event_date"].dropna()
events_covered = events.between(lbl_min, lbl_max).all() if len(events) else True

temporal_pass = within_timeline and events_covered

print("\n🕒 TEMPORAL ALIGNMENT")
print(f"- Transactions range: {tx_min.date()} → {tx_max.date()}")
print(f"- Labels range:       {lbl_min.date()} → {lbl_max.date()}")
print(f"- Tx within labels:   {within_timeline}")
print(f"- Events covered:     {events_covered}")
print(f"✅ Temporal Alignment: {'PASS' if temporal_pass else 'FAIL'}")



🕒 TEMPORAL ALIGNMENT
- Transactions range: 2025-02-16 → 2025-08-20
- Labels range:       2025-02-16 → 2025-08-20
- Tx within labels:   True
- Events covered:     True
✅ Temporal Alignment: PASS


In [4]:
# make 100% sure dates are datetime
transactions["date"] = pd.to_datetime(transactions["date"])
daily_labels["date"] = pd.to_datetime(daily_labels["date"])
outcomes["event_date"] = pd.to_datetime(outcomes["event_date"])

tx_min, tx_max = transactions["date"].min(), transactions["date"].max()
lbl_min, lbl_max = daily_labels["date"].min(), daily_labels["date"].max()
ev_min, ev_max = outcomes["event_date"].min(), outcomes["event_date"].max()

print("TX:", tx_min.date(), "→", tx_max.date())
print("LB:", lbl_min.date(), "→", lbl_max.date())
print("EV:", ev_min.date() if pd.notna(ev_min) else None, "→", ev_max.date() if pd.notna(ev_max) else None)

print("\nWhich condition failed?")
print("- tx_min < lbl_min:", tx_min < lbl_min)
print("- tx_max > lbl_max:", tx_max > lbl_max)
print("- any event < lbl_min:", outcomes["event_date"].dropna().lt(lbl_min).any())
print("- any event > lbl_max:", outcomes["event_date"].dropna().gt(lbl_max).any())


TX: 2025-02-16 → 2025-08-20
LB: 2025-02-16 → 2025-08-20
EV: 2025-05-08 → 2025-08-05

Which condition failed?
- tx_min < lbl_min: False
- tx_max > lbl_max: False
- any event < lbl_min: False
- any event > lbl_max: False


/var/folders/1t/zpc_3c3d039cn7bcqzxdmsz80000gn/T/ipykernel_57354/1894370959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_labels["date"] = pd.to_datetime(daily_labels["date"])


In [5]:
# Distress rate from outcomes
distress_rate = outcomes["is_distressed"].mean()

# Simple signal probe: avg high-risk spend 7 days before event vs. far-away period
tx_hr = transactions[transactions["category"].isin(["cash_advance","payday_loan"])]

# Join tx to outcomes to know customer event date
ev = outcomes[["customer_id","event_date"]].dropna()
tx_ev = tx_hr.merge(ev, on="customer_id", how="inner")

pre_mask = (tx_ev["date"] >= (tx_ev["event_date"] - pd.Timedelta(days=7))) & (tx_ev["date"] <= tx_ev["event_date"])
far_mask = (tx_ev["date"] <= (tx_ev["event_date"] - pd.Timedelta(days=30))) & (tx_ev["date"] >= (tx_ev["event_date"] - pd.Timedelta(days=60)))

pre_avg = tx_ev.loc[pre_mask, "amount"].abs().mean()
far_avg = tx_ev.loc[far_mask, "amount"].abs().mean()

signal_uplift = (pre_avg / far_avg) if (far_avg and not pd.isna(far_avg)) else float("nan")

# Heuristics for pass/fail
rate_ok = 0.05 <= distress_rate <= 0.30
signal_ok = (signal_uplift >= 1.15) if pd.notna(signal_uplift) else False  # at least +15%

label_pass = rate_ok and signal_ok

print("\n🏷️  LABEL QUALITY")
print(f"- Distress rate: {distress_rate:.3f}")
print(f"- High-risk spend pre-event vs. baseline (x): {signal_uplift:.2f}" if pd.notna(signal_uplift) else "- Not enough events to compute uplift")
print(f"✅ Label Quality: {'PASS' if label_pass else 'FAIL'}")



🏷️  LABEL QUALITY
- Distress rate: 0.192
- High-risk spend pre-event vs. baseline (x): 1.53
✅ Label Quality: PASS


## Final Result


In [6]:
all_checks = [integrity_pass, temporal_pass, label_pass]
score = sum(all_checks)

print(f"\n🎯 SANITY CHECK RESULTS: {score}/3 PASSED")
print("=" * 40)
if score == 3:
    print("🎉 EXCELLENT: Data ready for feature engineering!")
elif score == 2:
    print("✅ GOOD: Minor issues, proceed with caution")
else:
    print("❌ FAIL: Fix data issues before modeling")

print(f"\nDataset: {len(customers):,} customers, {len(transactions):,} transactions")
print("Ready for next step: Feature engineering")



🎯 SANITY CHECK RESULTS: 3/3 PASSED
🎉 EXCELLENT: Data ready for feature engineering!

Dataset: 2,500 customers, 1,122,312 transactions
Ready for next step: Feature engineering
